# Objects in boxes

## Objective
* We want to put $N$ objects into a row of $N$ boxes.

* Boxes are aligned from left to right (if $i < i'$, box $i$ is to the left of box $i'$) on the $x$ axis.
* Box $i$ is located at a point $B_i$ of the $(x,y)$ plane and object $j$ is located at $O_j$.

* We want to find an arrangement of objects such that:
    * each box contains exactly one object,
    * each object is stored in one box,
    * the total distance from object $j$ to its storage box is minimal.

* First, we solve the problem described, and then we add two new constraints and examine how the cost (and solution) changes.
    * From the first solution, we impose that object #1 is assigned to the box immediately to the left of object #2.
    * Then we impose that object #5 is assigned to a box next to the box of object #6.

In [26]:
import docplex.mp
from math import sqrt
import random

In [27]:
# Setup Data
N = 3 #8
box_range = range(1, N+1)
obj_range = range(1, N+1)

o_xmax = N*10
o_ymax = 2*N
box_coords = {b: (10*b, 1) for b in box_range}

obj_coords= {1: (140, 6), 2: (146, 8), 3: (132, 14), 4: (53, 28), 
             5: (146, 4), 6: (137, 13), 7: (95, 12), 8: (68, 9), 9: (102, 18), 
             10: (116, 8), 11: (19, 29), 12: (89, 15), 13: (141, 4), 14: (29, 4), 15: (4, 28)}

# the distance matrix from box i to object j - we compute the square of distance to keep integer
distances = {}
for o in obj_range:
    for b in box_range:
        dx = obj_coords[o][0]-box_coords[b][0]
        dy = obj_coords[o][1]-box_coords[b][1]
        d2 = dx*dx + dy*dy
        distances[b, o] = d2

In [28]:
# Create the Model
from docplex.mp.model import Model
mdl = Model("boxes")

#### Define the decision variables

* For each box $i$ ($i$ in $1..N$) and object $j$ ($j$ in $1..N$), we define a binary variable $X_{i,j}$ equal to $1$ if and only if object $j$ is stored in box $i$.

In [29]:
# decision variables is a 2d-matrix
x = mdl.binary_var_matrix(box_range, obj_range, lambda ij: "x_%d_%d" %(ij[0], ij[1]))

#### Express the business constraints

* The sum of $X_{i,j}$ over both rows and columns must be equal to $1$, resulting in $2\times N$ constraints.

In [30]:
# one object per box
mdl.add_constraints(mdl.sum(x[i,j] for j in obj_range) == 1
                   for i in box_range)
    
# one box for each object
mdl.add_constraints(mdl.sum(x[i,j] for i in box_range) == 1
                  for j in obj_range)

mdl.print_information()

Model: boxes
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 6
   - linear=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


#### Express the objective

* The objective is to minimize the total distance between each object and its storage box.

In [31]:
# minimize total displacement
mdl.minimize( mdl.sum(distances[i,j] * x[i,j] for i in box_range for j in obj_range) )

In [32]:
mdl.export_as_lp(path='800-Objects in Boxes.lp')

'800-Objects in Boxes.lp'

#### Solve the model


In [33]:
mdl.print_information()

assert mdl.solve(), "!!! Solve of the model fails"

Model: boxes
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 6
   - linear=6
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [34]:
mdl.report()
d1 = mdl.objective_value
#mdl.print_solution()

def make_solution_vector(x_vars):
    sol = [0]* N
    for i in box_range:
        for j in obj_range:
            if x[i,j].solution_value >= 0.5:
                sol[i-1] = j
                break
    return sol

def make_obj_box_dir(sol_vec):
    # sol_vec contains an array of objects in box order at slot b-1 we have obj(b)
    return { sol_vec[b]: b+1 for b in range(N)}
    
               
sol1 = make_solution_vector(x)
print("* solution: {0!s}".format(sol1))          

* model boxes solved with objective = 42983.000
* solution: [3, 1, 2]
